# Phi-4 Mini ONNX సమాంతర ఫంక్షన్ కాలింగ్ ట్యుటోరియల్

ఈ నోట్‌బుక్ Phi-4 Mini ను ONNX Runtime GenAI తో సమాంతర ఫంక్షన్ కాలింగ్ కోసం ఎలా ఉపయోగించాలో చూపిస్తుంది. ఫంక్షన్ కాలింగ్ మోడల్‌ను వినియోగదారుల అభ్యర్థనల ఆధారంగా బాహ్య టూల్స్ మరియు APIs ను తెలివిగా కాల్ చేయనిస్తుంది.

## అవలోకనం

ఈ ట్యుటోరియల్‌లో, మీరు నేర్చుకునే విషయాలు:
- Phi-4 Mini ను ONNX Runtime GenAI తో సెటప్ చేయడం
- విమానాలు మరియు హోటళ్లు బుకింగ్ కోసం ఫంక్షన్ స్కీమాలు నిర్వచించడం
- నిర్మిత అవుట్పుట్ కోసం Lark వ్యాకరణంతో మార్గనిర్దేశిత జనరేషన్ ఉపయోగించడం
- సంక్లిష్ట ప్రయాణ బుకింగ్ పరిస్థితుల కోసం సమాంతర ఫంక్షన్ కాల్స్ అమలు చేయడం

## ముందస్తు అవసరాలు

ఈ నోట్‌బుక్‌ను రన్ చేయడానికి ముందు, మీ వద్ద ఉండవలసినవి:
- Phi-4 Mini ONNX మోడల్‌ను డౌన్లోడ్ చేసుకున్నట్లు ఉండాలి
- `onnxruntime-genai` ప్యాకేజ్ ఇన్‌స్టాల్ చేసి ఉండాలి
- ఫంక్షన్ కాలింగ్ కాన్సెప్ట్‌లపై ప్రాథమిక అవగాహన


## దశ 1: అవసరమైన లైబ్రరీలు దిగుమతి చేయండి

ముందుగా, మనం ఫంక్షన్ కాలింగ్ అమలుకు అవసరమైన లైబ్రరీలను దిగుమతి చేసుకుంటాము.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## దశ 2: మోడల్ సెటప్ మరియు కాన్ఫిగరేషన్

ఇప్పుడు మేము Phi-4 Mini ONNX మోడల్‌ను కాన్ఫిగర్ చేస్తున్నాము. దయచేసి path ని మీ నిజమైన మోడల్ డైరెక్టరీతో తప్పకుండా మార్చండి.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## దశ 3: జనరేషన్ పరామితులను కాన్ఫిగర్ చేయండి

మోడల్ యొక్క అవుట్‌పుట్ ప్రవర్తనను నియంత్రించడానికి జనరేషన్ పరామితులను సెట్ చేయండి. ఈ సెట్టింగ్‌లు ఫంక్షన్ కాలింగ్ కోసం నిర్ణీతమైన మరియు దృష్టి-కేంద్రీకృత ప్రతిస్పందనలను నిర్ధారిస్తాయి.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## దశ 4: అందుబాటులో ఉన్న ఫంక్షనులను నిర్వచించండి

ఇక్కడ మన AI అసిస్టెంట్ పిలవగల ఫంక్షన్లను నిర్వచిస్తాము. ఈ ఉదాహరణలో, మనకు ప్రయాణ సంబంధిత రెండు ఫంక్షన్లు ఉన్నాయి:
1. **booking_flight_tickets**: ఎయిర్‌పోర్టుల మధ్య విమానాల బుకింగ్ కోసం
2. **booking_hotels**: హోటల్ వసతుల బుకింగ్ కోసం

ఫంక్షన్ నిర్వచనాలు OpenAI యొక్క ఫంక్షన్-కాలింగ్ స్కీమా ఫార్మాట్‌ను అనుసరిస్తాయి.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## దశ 5: వ్యాకరణ ఉత్పత్తి సహాయక ఫంక్షన్లు

ఈ సహాయక ఫంక్షన్లు మా ఫంక్షన్ నిర్వచనాలను Lark వ్యాకరణ ఫార్మాట్‌కి మార్చుతాయి, ఇది ONNX Runtime GenAI ద్వారా మార్గనిర్దేశిత జనరేషన్ కోసం ఉపయోగించబడుతుంది. ఇది మోడల్ సరైన JSON ఫార్మాట్‌లో చెల్లుబాటు అయ్యే ఫంక్షన్ కాల్స్‌ను ఉత్పత్తి చేయాలని నిర్ధారిస్తుంది.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## దశ 6: వ్యాకరణ ఉత్పత్తిని పరీక్షించండి

మన టూల్ నిర్వచనాలను సరైన ఫార్మాట్‌కు ఎలా మార్చుతున్నాయో చూడటానికి మన వ్యాకరణ ఉత్పత్తి ఫంక్షన్లను పరీక్షిద్దాం.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## దశ 7: సిస్టమ్ ప్రాంప్ట్ మరియు జనరేటర్ సిద్ధం చేయండి

ఇప్పుడు మేము అందుబాటులో ఉన్న సాధనాల గురించి మోడల్‌కు తెలియజేసే సిస్టమ్ ప్రాంప్ట్‌ను సృష్టించి, గైడ్ చేసిన జనరేషన్ పరామితులతో జనరేటర్‌ను సెటప్ చేస్తాము.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## దశ 8: మార్గనిర్దేశిత జనరేషన్‌తో జనరేటర్‌ను ప్రారంభించండి

ఇప్పుడు మనం మా కాన్ఫిగర్ చేసిన పారామితులతో జనరేటర్‌ను సృష్టించి, మార్గనిర్దేశిత జనరేషన్ కోసం Lark వ్యాకరణాన్ని వర్తింపజేస్తాము.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## దశ 9: సమాంతర ఫంక్షన్ పిలవడం పరీక్షించండి

ఇప్పుడు మన సెటప్‌ను బహుళ ఫంక్షన్లు పిలవాల్సిన సంక్లిష్ట ప్రయాణ బుకింగ్ సన్నివేశంతో పరీక్షిద్దాం.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### ఫార్మాట్ చేయబడిన వినియోగదారు ప్రాంప్ట్‌ను వీక్షించండి


### ఫంక్షన్ కాల్స్ జనరేట్ చేయండి

మోడల్ ఇప్పుడు మా వినియోగదారు అభ్యర్థన ఆధారంగా నిర్మితమైన స్ట్రక్చర్డ్ ఫంక్షన్ కాల్‌లను తయారుచేస్తుంది. మార్గదర్శక జనరేషన్ కారణంగా అవుట్‌పుట్ నేరుగా అమలు చేయగలిగే సరైన JSON ఫార్మాట్‌లో ఉంటుంది.

**అంచనా అవుట్‌పుట్**: మోడల్ క్రింది ఫంక్షన్ کال్స్‌ను ఉత్పత్తి చేయాలి:
1. `booking_flight_tickets` Beijing (PEK) నుండి Paris (CDG) కి సంబంధించిన వివరాలు
2. `booking_hotels` Paris లోని నివాస సంబంధి వివరాలతో

కింద ఉన్న సెల్‌ను నడిపి ప్రత్యక్ష జనరేషన్‌ను చూడండి:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## సంక్షేపం

🎉 **అభినందనలు!** మీరు ONNX Runtime GenAI ఉపయోగించి Phi-4 Mini తో సమాంతర ఫంక్షన్ కాలింగ్‌ను విజయవంతంగా అమలు చేశారు.

### మీరు నేర్చుకున్నవి:

1. **మోడల్ సెటప్**: Phi-4 Miniని ONNX Runtime GenAIతో ఎలా కాన్ఫిగర్ చేయాలో
2. **ఫంక్షన్ నిర్వచనం**: AI ఫంక్షన్ కాలింగ్ కోసం ఫంక్షన్ స్కీమాలను ఎలా నిర్వచించాలో
3. **గైడెడ్ జనరేషన్**: నిర్మాణాత్మక అవుట్పుట్ జనరేషన్ కోసం Lark వ్యాకరణాన్ని ఎలా ఉపయోగించాలో
4. **సమాంతర ఫంక్షన్ కాల్స్**: బహుళ ఫంక్షన్ కాల్స్ అవసరమయ్యే సంక్లిష్ట అభ్యర్థనలను ఎలా నిర్వహించాలో

### ముఖ్య ప్రయోజనాలు:

- ✅ **సంరచిత అవుట్‌పుట్**: మార్గదర్శక జనరేషన్ చెల్లుబాటు అయ్యే JSON ఫంక్షన్ కాల్స్‌ను నిర్ధారిస్తుంది
- ✅ **సమాంతర ప్రాసెసింగ్**: ఒకే అభ్యర్థనలో బహుళ ఫంక్షన్ కాల్స్‌ను నిర్వహిస్తుంది
- ✅ **ఉన్నత పనితీరు**: ONNX Runtime ఆప్టిమైజ్డ్ ఇన్ఫరెన్స్‌ను అందిస్తుంది
- ✅ **అనుకూల స్కీమా**: ఫంక్షన్ నిర్వచనాలను జోడించడం లేదా మార్చడం సులభం

### వనరులు:

- [Phi-4 Mini డాక్యుమెంటేషన్](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI డాక్యుమెంటేషన్](https://onnxruntime.ai/docs/genai/)
- [ఫంక్షన్ కాలింగ్ ఉత్తమ పద్ధతులు](https://platform.openai.com/docs/guides/function-calling)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
డిస్క్లెయిమర్:
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించగా కూడా, స్వయంచాలక అనువాదాల్లో తప్పులు లేదా లోపాలు ఉండవచ్చు. స్థానిక భాషలో ఉండే అసలు పత్రాన్ని అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారం కోసం వృత్తిపరమైన మానవ అనువాదాన్ని సిఫార్సు చేయబడుతుంది. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుగా అర్థం చేసుకోవడాలకై మేము బాధ్యులు కాదు.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
